## 1. Bibliotecas e módulos úteis

In [1]:
import re
import pandas as pd

import spacy

In [2]:
spacy.__version__

'3.0.5'

## 2. Carregando as bases de dados relacionadas ao BC2GM(BioCreative II Gene Mention) _Corpus_:

> [Gene Mention Tagging task](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2008-9-s2-s2) as part of the BioCreative II challenge is concerned with the named entity extraction of gene and gene product mentions in text. The BC2GM corpus contains a total of 24,583 gene entity mentions.

--- 

* Os aquivos estão no formato `.tsv` (_tab-separated values_). Podemos utilizar tanto o método `pd.read_table()` quanto o `pd.read_csv()` para carregá-los;
  Nesse caso, é necessário especificar o separador `sep='\t'`, indicando se tratar de tabulacão; ou o _alias_ para esse parâmetro, `delimiter='\t'`;
* Os arquivos não possuem cabeçalho. Assim, apenas para melhor compreensão dos dados, nomeamos as colunas por "word" e "tag" por meio do parâmetro `names=['word', 'tag']`;
* Ao tentarmos carregar o arquivo de treino, ocorreu o seguinte erro - acreditomos que devido ao seu tamanho:

    ```python
    Error tokenizing data. C error: EOF inside string starting at row 131598
    ```
* A solução encontrada foi utilizar o parâmetro `quoting=3`.

In [3]:
# Carrega a base de treino e validação(dev) juntas
bc2gm_train = pd.read_csv('./datasets/BC2GM/train_dev.tsv', sep='\t', names=['word', 'tag'], quoting=3)

# Carrega a base de teste
bc2gm_test = pd.read_csv('./datasets/BC2GM/test.tsv', sep='\t', names=['word', 'tag'])

## 2.1 Análise Exploratória de Dados

In [4]:
# Exibe as primeiras linhas da base de treino
bc2gm_train.head()

,word,tag
0,Immunohistochemical,O
1,staining,O
2,was,O
3,positive,O
4,for,O


In [5]:
# Exibe as linhas iniciais da base de teste
bc2gm_test.head()

,word,tag
0,Physical,O
1,mapping,O
2,220,O
3,kb,O
4,centromeric,O


In [6]:
# Exibe a distibuição de valores para a coluna "tag" na base de treino
bc2gm_train['tag'].value_counts(dropna=False)

O    381648
I     26541
B     18258
Name: tag, dtype: int64

In [7]:
# Exibe a distibuição de valores para a coluna "tag" na base de treino
bc2gm_test['tag'].value_counts(dropna=False)

O    98965
I     6694
B     4765
Name: tag, dtype: int64

De acordo com a coluna `tag`, temos uma típica represetação para tarefas de linguística computacional: O formato __IOB__ - um acrônimo para _Inside_, _Outside_, _Beginning_.

[Inside–outside–beginning (tagging)](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging))

In [8]:
print(f'A base de treino possui {bc2gm_train.shape[0]:,} linas e {bc2gm_train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de teste possui {bc2gm_test.shape[0]:,} linnas e {bc2gm_test.shape[1]:,} colunas'.replace(',', '.'))

A base de treino possui 426.447 linas e 2 colunas
A base de teste possui 110.424 linnas e 2 colunas


## 2.2. Procurando por Dados Faltantes

In [9]:
# Verifica se há missing values na base de treino
print('Valores faltantes na base de treino:' , '-' * 37, bc2gm_train.isna().sum(), sep='\n', end='\n\n')

# Verifica se há missing values na base de test
print('Valores faltantes na base de teste:', '-' * 37, bc2gm_test.isna().sum(), sep='\n')

Valores faltantes na base de treino:
-------------------------------------
word    26
tag      0
dtype: int64

Valores faltantes na base de teste:
-------------------------------------
word    9
tag     0
dtype: int64


In [10]:
# Remove dados faltantes da base de treino e teste
bc2gm_train.dropna(inplace=True)
bc2gm_test.dropna(inplace=True)

# Sanity check
print(f'A base de treino possui {bc2gm_train.shape[0]:,} linas e {bc2gm_train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de teste possui {bc2gm_test.shape[0]:,} linnas e {bc2gm_test.shape[1]:,} colunas'.replace(',', '.'))

A base de treino possui 426.421 linas e 2 colunas
A base de teste possui 110.415 linnas e 2 colunas


## 3 Carregando as bases de dados relacionadas ao BC4CHEMD Corpus:

In [11]:
# Carrega a base de treino e validação(dev) juntas
bc4chemd_train = pd.read_csv('./datasets/BC4CHEMD/train_dev.tsv', sep='\t', names=['word', 'tag'], quoting=3)

# Carrega a base de teste
bc4chemd_test = pd.read_csv('./datasets/BC4CHEMD/test.tsv', sep='\t', names=['word', 'tag'], quoting=3)

## 3.1 Análise Exploratória de Dados

In [12]:
# Exibe as primeiras linhas da base de treino
bc4chemd_train.head()

,word,tag
0,DPP6,O
1,as,O
2,a,O
3,candidate,O
4,gene,O


In [13]:
# Exibe as linhas iniciais da base de teste
bc4chemd_test.head()

,word,tag
0,Effects,O
1,of,O
2,docosahexaenoic,B
3,acid,I
4,and,O


## 3.2. Procurando por Dados Faltantes

In [14]:
# Verifica se há missing values na base de treino
print('Valores faltantes na base de treino:' , '-' * 37, bc4chemd_train.isna().sum(), sep='\n', end='\n\n')

# Verifica se há missing values na base de test
print('Valores faltantes na base de teste:', '-' * 37, bc4chemd_test.isna().sum(), sep='\n')

Valores faltantes na base de treino:
-------------------------------------
word    95
tag      0
dtype: int64

Valores faltantes na base de teste:
-------------------------------------
word    20
tag      0
dtype: int64


In [15]:
# Remove dados faltantes da base de treino e teste
bc4chemd_train.dropna(inplace=True)
bc4chemd_test.dropna(inplace=True)

# Sanity check
print(f'A base de treino possui {bc4chemd_train.shape[0]:,} linas e {bc4chemd_train.shape[1]:,} colunas')
print(f'A base de teste possui {bc4chemd_test.shape[0]:,} linnas e {bc4chemd_test.shape[1]:,} colunas')

A base de treino possui 1,781,395 linas e 2 colunas
A base de teste possui 767,616 linnas e 2 colunas


## 4. Criação dos arquivos para treinamento do modelo por meio do spaCy
---
Na versão mais recente do spaCy, a transformação de arquivos para o formato `.spacy` é feita via linha de comando.   
É necessario um arquivo `config.cfg` para se utilizar a linha de comando do spaCy.    
Esse arquivo pode ser criado [aqui](https://spacy.io/usage/training#quickstart).    
Daí, basta clicar no ícone de download, salvar o arquivo na pasta do projeto e executar a seguinte linha de comando:
```bash
$ python -m spacy init fill-config base_config.cfg config.cfg
```

#### 4.1. Exibição no _prompt_ após execução da linha de comando acima:
<img src="assets/config.PNG" width=60%>

#### 4.2. As configurações por mim escolhidas: 


<img src="assets/quickstart.png" width=70%>


Se tentarmos gerar os arquivos `.spacy` utilizando somente as _tags_ IOB, será gerado um erro.   
É preciso completar a _tag_ para o __I__ e o __B__.   
Utilizaremos a _tag_ disponível na tabela encaminhada pelo professor Cristian.

In [16]:
bc2gm_train['tag'].replace({'I': 'I-DRUG-PROTEIN', 'B': 'B-DRUG-PROTEIN'}, inplace=True)
bc2gm_test['tag'].replace({'I': 'I-DRUG-PROTEIN', 'B': 'B-DRUG-PROTEIN'}, inplace=True)

bc4chemd_train['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)
bc4chemd_test['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)

In [17]:
print(bc2gm_train['tag'].unique(), bc2gm_test['tag'].unique(), bc4chemd_train['tag'].unique(), bc4chemd_test['tag'].unique(), sep='\n')

['O' 'B-DRUG-PROTEIN' 'I-DRUG-PROTEIN']
['O' 'B-DRUG-PROTEIN' 'I-DRUG-PROTEIN']
['O' 'B-CHEMICALS' 'I-CHEMICALS']
['O' 'B-CHEMICALS' 'I-CHEMICALS']


In [18]:
# Base de dados reduzida, somente para exemplificação
train = pd.concat([bc2gm_train, bc4chemd_train])
# test = pd.concat([bc2gm_test, bc4chemd_test])
test = pd.concat([bc2gm_test, bc4chemd_test])


print(f'A base de dados gerada para treino posssui {train.shape[0]:,} linhas e {train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de dados gerada para teste possui {test.shape[0]:,} linhas e {test.shape[1]:,} colunas'.replace(',', '.'))

A base de dados gerada para treino posssui 2.207.816 linhas e 2 colunas
A base de dados gerada para teste possui 878.031 linhas e 2 colunas


In [19]:
# Remove dados que impedem a conversão dos arquivos
pattern = r'\b\t\w*'
mask = test['word'].str.contains(pattern, regex=True)
print('Quantidade de linhas que serão removidas:', test[mask].shape[0])
test[mask].head()

Quantidade de linhas que serão removidas: 39


,word,tag
10012,\tO\ntranscription\tO\n,O
18862,\tO\nequivalent\tO\n,O
20405,\tO\nUp\tO\nand\tO\nGo\tO\n,O
22266,\tO\nMudzez\tO\nal\tO\n-\tO\nKanum\tO\n,O
22289,\tO\nBibliografija\tO\nmedicinskih\tO\ndjela\t...,O


In [20]:
test = test[mask == False].copy()

print(f'A base de dados gerada para treino posssui {train.shape[0]:,} linhas e {train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de dados gerada para teste possui {test.shape[0]:,} linhas e {test.shape[1]:,} colunas'.replace(',', '.'))

A base de dados gerada para treino posssui 2.207.816 linhas e 2 colunas
A base de dados gerada para teste possui 877.992 linhas e 2 colunas


In [21]:
print(train['tag'].unique(), test['tag'].unique(), sep='\n')

['O' 'B-DRUG-PROTEIN' 'I-DRUG-PROTEIN' 'B-CHEMICALS' 'I-CHEMICALS']
['O' 'B-DRUG-PROTEIN' 'I-DRUG-PROTEIN' 'B-CHEMICALS' 'I-CHEMICALS']


## 5. Salvando os arquivos pré-processados

In [22]:
train.to_csv('./datasets/spacy/train.iob', sep='\t', header=0, index=0)
test.to_csv('./datasets/spacy/test.iob', sep='\t', header=0, index=0)

In [23]:
pd.read_csv('./datasets/spacy/train.iob', sep='\t', names=['word', 'tag'])['tag'].value_counts(dropna=False)

O                 2033271
I-CHEMICALS         70786
B-CHEMICALS         58964
I-DRUG-PROTEIN      26537
B-DRUG-PROTEIN      18258
Name: tag, dtype: int64

In [24]:
pd.read_csv('./datasets/spacy/test.iob', sep='\t', names=['word', 'tag'])['tag'].value_counts(dropna=False)

O                 811549
I-CHEMICALS        29642
B-CHEMICALS        25346
I-DRUG-PROTEIN      6691
B-DRUG-PROTEIN      4764
Name: tag, dtype: int64

## 6. Convertendo os arquivos `.iob` para o formato `.spacy`
---
Utilizaremos o exemplo disponível no [GitHub do spaCy](https://github.com/explosion/spaCy/tree/master/extra/example_data/ner_example_data)

```bash
$ python -m spacy convert -c iob -s -n 10 -b en_core_web_sm file.iob .
```

**Obs.:** Removemos a opção `-b en_core_web_sm` pois o modelo não estava convergindo com essa opção

Salvaremos os arquivos convertidos em um diretório chamado _datasets/spacy_:
```python
# Converte o arquivo contendo a base de treino
$ python -m spacy convert -c iob -s -n 10 ./datasets/spacy/train.iob ./datasets/spacy

# Converte o arquivo contendo a base de test
$ python -m spacy convert -c iob -s -n 10 ./datasets/spacy/test.iob ./datasets/spacy
```

#### Saída no _prompt_ de comando:

<img src="./assets/iob_para_spacy.png" width=60%>

```python
# Converte o arquivo contendo a base de test
$ python -m spacy debug data config.cfg
```
#### Saída no _prompt_ de comando :

<img src="./assets/consistencia_arquivos.png" width=50%>

## 7. Treinamento e Validação do modelo
---
Utilizando o template disponível em [Quickstart](https://spacy.io/usage/training#quickstart)

```bash
$ python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy
```

Para selecionar a GPU usamos a opção `--gpu-id`:
```bash
$ python -m spacy train config.cfg --gpu-id 0
```

Nossa linha de comando para treinamento e validação do modelo:

```bash
$ python -m spacy train config.cfg --gpu-id 0 --output ./datasets/spacy --paths.train ./datasets/spacy/train.spacy --paths.dev ./datasets/spacy/test.spacy
```

#### Saída no _prompt_ de comando:

<img src="./assets/treinamento.png" width=60%>



## 8. Inferência

In [25]:
# Carrega base de dados
corona_dataset = pd.read_csv('./datasets/corona_dataset.csv')

# Remove colunas sem desnecessárias
corona_dataset.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis='columns', inplace=True)

# Remove dados faltantes
corona_dataset.dropna(inplace=True)

# Exibe as linhas inicias da base de dados
corona_dataset.head()

,date,title,category,body,source
0,2020-03-19,Nets' Access to COVID-19 Tests Questioned,Pro Basketball,New York Mayor Bill de Blasio is among those q...,By Reuters
1,March 14,Ireland Announces Second Death From COVID-19,Europe,A second patient has died of the COVID-19 viru...,By Reuters
2,March 6,England to Register COVID-19 as 'Notifiable Di...,Europe,"England will formally register COVID-19, a dis...",By Reuters
3,March 18,"Lakers Reportedly Under Quarantine, Will Test ...",Pro Basketball,Members of the Los Angeles Lakers are under qu...,By Reuters
4,March 17,Factbox: COVID-19 and the New Coronavirus-Fact...,Europe,Social media is awash with myths about how peo...,By Reuters


In [26]:
corpus = ' '.join(corona_dataset['body'])

# Carrega o modelo customizado e cria um objeto nlp
nlp = spacy.load('./datasets/spacy/model-best')

# Processa o texto
doc = nlp(corpus)

In [27]:
for token in doc.ents:
    print(f'{token.text:<20} {token.label_:>10}')

outlets               CHEMICALS
Indonesian            CHEMICALS
Saturday              CHEMICALS
”                     CHEMICALS
”                     CHEMICALS
charities             CHEMICALS
ATP                   CHEMICALS
WTA                   CHEMICALS
business              CHEMICALS
hotels                CHEMICALS
U. A                  CHEMICALS
ATP                   CHEMICALS
Co                    CHEMICALS
Co                    CHEMICALS
”                     CHEMICALS
coronavirus           CHEMICALS
cancel                CHEMICALS
saying                CHEMICALS
rugby                 CHEMICALS
Co                    CHEMICALS
nitrogen dioxide      CHEMICALS
Sunday                CHEMICALS
revamp                CHEMICALS
Sunday                CHEMICALS
saying                CHEMICALS
deaths                CHEMICALS
Sunday                CHEMICALS
Eli                   CHEMICALS
Co                    CHEMICALS
COVID-19              CHEMICALS
saying                CHEMICALS
coronavi

In [28]:
spacy.displacy.render(doc[:332], style="ent", jupyter=True)

Referências:

[SpaCy](https://spacy.io/)   
[Building a custom NER model in Spacy v3.1](https://zachlim98.github.io/me/2021-03/spacy3-ner-tutorial)   
[NER @ CLI: Custom-named entity recognition with spaCy in four lines](https://blog.codecentric.de/en/2020/11/ner-cli-custom-named-entity-recognition-with-spacy-in-four-lines/)   
[Descubre Named Entity Recognition (NER) con Spacy 3.0 (en castellano)](https://nymiz.com/blog/tecnologia/que-es-el-named-entity-recognition-ner/)




$ python -m spacy debug data config.cfg